# 데이터 전처리
* **spaCy 라이브러리** : 문장의 토큰화, 태깅 등 전처리 기능을 위한 라이브러리

In [6]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [7]:
import spacy

spacy_en = spacy.load('en') # 영어 토큰화
spacy_de = spacy.load('de') # 독일어 토큰화

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

# 일반적인 데이터 로더의 iterator와 유사하기 사용
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size = BATCH_SIZE,
    device=device
)

NameError: name 'BucketIterator' is not defined

# 인코더 아키텍처
* 주어진 소스 문장을 문맥 벡터(context vecotr)로 인코딩한다.
* LSTM은 hidden state와 cell state를 반환한다
* 하이퍼 파라미터
    * input_dim
    * embed_dim
    * hidden_dim
    * n_layers
    * dropout_ratio

In [4]:
import torch.nn as nn

# 인코더(Encoder 아키텍처 정의)
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()
        
        # 임베딩은 원-핫 인코딩을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)
        
        # LSTM
        self.hidden_dm = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=droput_ratio)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)
        
    # 인코드는 src 문장을 입력받아 문맥 벡터를 반환
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        return hidden,cell

ModuleNotFoundError: No module named 'torch'

# 디코더 아키텍처
* encoder에서 주어진 문맥 벡터를 타겟 문장으로 디코딩한다
* LSTM은 hidden state와 cell state를 반환한다
* 하이퍼 파라미터는 인코더와 동일함

In [5]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()
        
        self.embedding = nn.Embedding(output_dim, embed_dim)
        
        # LSTM
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        
        # FC 
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
        # 드롭아웃
        self.dropout = nn.Dropout(dropout_ratio)
        
    def forward(self, input, hidden, cell):
        # input : [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        # 단어 개수는 어차피 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        
        # 현재 출력 단어, 현재까지의 모든 정보, 모든 단어의 정보
        return prediction, hidden, cell

NameError: name 'nn' is not defined

# Seq2Seq 아키텍처
단, 디코더는 한 단어씩 넣어서 한 번씩 결과를 구한다.
* Teacher forcing : 디코더의 prediction을 다음 입력으로 사용하지 않고, 실제 목표 출력을 다음입력으로 사용하는 기법

In [6]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    # 학습 시, 소스 문장, 타겟 문장, teacher forcing ratio 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # 먼저 인코더를 거쳐 문맥 벡터를 추출
        hidden, cell = self.encoder(src)
        
        # 디코더의 최종 결과를 담을 Tensor 객체 생성
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim 
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # 첫번째 입력은 항상 <sos> 토큰
        input = trg[0, :]
        
        # 타겟 단어의 개수만큼 반복하여 디코더에 forwarding
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            outputs[t] = output
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출
            
            # teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재 출력 결과를 다음 input에 넣기
        
        return ouptus

NameError: name 'nn' is not defined

# 학습 (training)
* 하이퍼 파라미터 설정 및 모델 초기화

In [7]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

NameError: name 'SRC' is not defined

In [8]:
# 인코더와 디코더 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

NameError: name 'Encoder' is not defined

* 논문의 내용대로 모델 가중치 파라미터 초기화

In [9]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

NameError: name 'model' is not defined

* 학습 및 평가 함수 정의

In [10]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분 padding에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

ModuleNotFoundError: No module named 'torch'

In [11]:
# 모델 학습 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    
    # 전체 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        output_dim = output.shape[-1] # 출력 단어 개수, 배치 크기, 출력 차원
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        # 기울기 clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 계산
        epoch_loss += loss.item()
        
    return epoch_loss/len(iterator)

* 학습 및 검증 진행
    * 학습 횟수 (epoch) : 20

In [13]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시간 기록
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time() # 종료 시간
    epoch_mins, epoch_sess = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')
        
    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

NameError: name 'model' is not defined

In [ ]:
s